In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna
!pip install catboost

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, spearmanr, skew, pearsonr, zscore

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import KNeighborsClassifier

#SMOTE
from imblearn.over_sampling import SMOTE

#Mô hình XG Boost
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import RandomizedSearchCV

#Random Forest
from sklearn.ensemble import RandomForestClassifier

import joblib
import optuna
from catboost import CatBoostClassifier

In [38]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/Credit Card Fraud Dataset/application_data.csv')

# **GIẢI THÍCH DỮ LIỆU:**

SK_ID_CURR: Mã định danh của khách hàng. Đây là một giá trị duy nhất cho mỗi khách hàng trong tập dữ liệu.

**TARGET:** Biến mục tiêu, thường là một giá trị nhị phân (0 hoặc 1). Trong các bài toán phân loại rủi ro tín dụng, giá trị 1 có thể đại diện cho việc khách hàng không trả được nợ (default), trong khi 0 cho thấy khách hàng đã trả nợ đúng hạn.

NAME_CONTRACT_TYPE: Loại hợp đồng tín dụng (ví dụ: "Cash loans" hoặc "Revolving loans"). Cột này cho biết loại sản phẩm tín dụng mà khách hàng đang vay.

CODE_GENDER: Mã giới tính của khách hàng (ví dụ: "M" cho nam, "F" cho nữ).

FLAG_OWN_CAR: Cờ cho biết khách hàng có sở hữu xe hơi không (1 nếu có, 0 nếu không).

FLAG_OWN_REALTY: Cờ cho biết khách hàng có sở hữu bất động sản không (1 nếu có, 0 nếu không).

CNT_CHILDREN: Số lượng con cái của khách hàng.

AMT_INCOME_TOTAL: Tổng thu nhập hàng năm của khách hàng.

AMT_CREDIT: Số tiền tín dụng mà khách hàng đã vay.

AMT_ANNUITY: Số tiền thanh toán hàng năm mà khách hàng phải trả.

AMT_GOODS_PRICE: Giá trị hàng hóa hoặc dịch vụ mà khách hàng đang mua bằng tín dụng.

NAME_TYPE_SUITE: Loại nhóm hoặc hộ gia đình mà khách hàng thuộc về (ví dụ: "Unaccompanied", "Family", "Spouse, partner", etc.).

NAME_INCOME_TYPE: Loại thu nhập của khách hàng (ví dụ: "Working", "Commercial associate", "Pensioner", etc.).

NAME_EDUCATION_TYPE: Trình độ học vấn của khách hàng (ví dụ: "Higher education", "Secondary education", "Incomplete higher", etc.).

NAME_FAMILY_STATUS: Tình trạng hôn nhân của khách hàng (ví dụ: "Single", "Married", "Widow", etc.).

NAME_HOUSING_TYPE: Loại hình cư trú của khách hàng (ví dụ: "House / apartment", "Rented apartment", "With parents", etc.).

REGION_POPULATION_RELATIVE: Tỷ lệ dân số của khu vực mà khách hàng đang sinh sống so với tổng dân số.

DAYS_BIRTH: Tuổi của khách hàng tính bằng số ngày kể từ ngày sinh đến thời điểm hiện tại. Thường là số âm (số ngày từ ngày sinh đến ngày hiện tại).

DAYS_EMPLOYED: Số ngày khách hàng đã làm việc kể từ ngày bắt đầu công việc đến thời điểm hiện tại. Thường là số âm (số ngày từ ngày bắt đầu công việc đến ngày hiện tại).

DAYS_REGISTRATION: Số ngày kể từ khi khách hàng đăng ký cư trú đến thời điểm hiện tại. Thường là số âm.

DAYS_ID_PUBLISH: Số ngày kể từ khi khách hàng đăng ký chứng minh thư đến thời điểm hiện tại. Thường là số âm.

OWN_CAR_AGE: Tuổi của chiếc xe hơi mà khách hàng sở hữu (số năm). Có thể là giá trị số hoặc thiếu nếu không sở hữu xe.

FLAG_MOBIL: Cờ cho biết khách hàng có số điện thoại di động không (1 nếu có, 0 nếu không).

FLAG_EMP_PHONE: Cờ cho biết khách hàng có số điện thoại công ty không (1 nếu có, 0 nếu không).

FLAG_WORK_PHONE: Cờ cho biết khách hàng có số điện thoại công việc không (1 nếu có, 0 nếu không).

FLAG_CONT_MOBILE: Cờ cho biết khách hàng có thể liên hệ qua di động không (1 nếu có, 0 nếu không).

FLAG_PHONE: Cờ cho biết khách hàng có số điện thoại không (1 nếu có, 0 nếu không).

FLAG_EMAIL: Cờ cho biết khách hàng có email không (1 nếu có, 0 nếu không).

OCCUPATION_TYPE: Loại nghề nghiệp của khách hàng (ví dụ: "Laborers", "Sales staff", "Managers", etc.).

CNT_FAM_MEMBERS: Số lượng thành viên trong gia đình của khách hàng.

REGION_RATING_CLIENT: Xếp hạng của khu vực mà khách hàng sống (thường từ 1 đến 3, với 1 là khu vực ít phát triển hơn).

REGION_RATING_CLIENT_W_CITY: Xếp hạng khu vực, tính đến cả thành phố mà khách hàng sống.

WEEKDAY_APPR_PROCESS_START: Ngày trong tuần khi quá trình phê duyệt bắt đầu (ví dụ: "Monday", "Tuesday", etc.).

HOUR_APPR_PROCESS_START: Giờ trong ngày khi quá trình phê duyệt bắt đầu.

REG_REGION_NOT_LIVE_REGION: Cờ cho biết khu vực đăng ký không phải là khu vực khách hàng sống (1 nếu đúng, 0 nếu không).

REG_REGION_NOT_WORK_REGION: Cờ cho biết khu vực đăng ký không phải là khu vực khách hàng làm việc (1 nếu đúng, 0 nếu không).

LIVE_REGION_NOT_WORK_REGION: Cờ cho biết khu vực khách hàng sống không phải là khu vực làm việc (1 nếu đúng, 0 nếu không).

REG_CITY_NOT_LIVE_CITY: Cờ cho biết thành phố đăng ký không phải là thành phố khách hàng sống (1 nếu đúng, 0 nếu không).

REG_CITY_NOT_WORK_CITY: Cờ cho biết thành phố đăng ký không phải là thành phố khách hàng làm việc (1 nếu đúng, 0 nếu không).

LIVE_CITY_NOT_WORK_CITY: Cờ cho biết thành phố khách hàng sống không phải là thành phố khách hàng làm việc (1 nếu đúng, 0 nếu không).

ORGANIZATION_TYPE: Loại tổ chức mà khách hàng làm việc (ví dụ: "Business Entity", "Government", "School", etc.).

EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3: Các nguồn dữ liệu bên ngoài (ví dụ: điểm số tín dụng từ các tổ chức bên ngoài). Thường là các chỉ số được tính toán từ dữ liệu bên ngoài để đánh giá khả năng tín dụng.

APARTMENTS_AVG: Diện tích trung bình của các căn hộ trong khu vực cư trú.

BASEMENTAREA_AVG: Diện tích trung bình của tầng hầm trong khu vực cư trú.

YEARS_BEGINEXPLUATATION_AVG: Thời gian trung bình kể từ khi bất động sản bắt đầu được sử dụng.

YEARS_BUILD_AVG: Tuổi trung bình của các tòa nhà trong khu vực cư trú.

COMMONAREA_AVG: Diện tích trung bình của các khu vực chung trong bất động sản.

ELEVATORS_AVG: Số lượng thang máy trung bình trong khu vực cư trú.

ENTRANCES_AVG: Số lượng lối vào trung bình trong khu vực cư trú.

FLOORSMAX_AVG: Số tầng tối đa trung bình của các tòa nhà trong khu vực cư trú.

FLOORSMIN_AVG: Số tầng tối thiểu trung bình của các tòa nhà trong khu vực cư trú.

LANDAREA_AVG: Diện tích đất trung bình trong khu vực cư trú.

LIVINGAPARTMENTS_AVG: Số lượng căn hộ sinh sống trung bình trong khu vực cư trú.

LIVINGAREA_AVG: Diện tích sinh sống trung bình trong khu vực cư trú.

NONLIVINGAPARTMENTS_AVG: Số lượng căn hộ không sinh sống trung bình trong khu vực cư trú.

NONLIVINGAREA_AVG: Diện tích không sinh sống trung bình trong khu vực cư trú.

APARTMENTS_MODE: Chế độ số lượng căn hộ trong khu vực cư trú. Đây là số lượng căn hộ xuất hiện nhiều nhất trong các bất động sản trong khu vực.

BASEMENTAREA_MODE: Chế độ diện tích tầng hầm của các bất động sản. Đây là diện tích tầng hầm phổ biến nhất.

YEARS_BEGINEXPLUATATION_MODE: Chế độ số năm kể từ khi bất động sản bắt đầu được sử dụng. Đây là giá trị phổ biến nhất trong số các bất động sản.

YEARS_BUILD_MODE: Chế độ tuổi của các tòa nhà trong khu vực cư trú. Đây là số năm xây dựng phổ biến nhất.

COMMONAREA_MODE: Chế độ diện tích khu vực chung trong các bất động sản. Đây là diện tích khu vực chung phổ biến nhất.

ELEVATORS_MODE: Chế độ số lượng thang máy trong các tòa nhà. Đây là số lượng thang máy phổ biến nhất.

ENTRANCES_MODE: Chế độ số lượng lối vào trong các tòa nhà. Đây là số lượng lối vào phổ biến nhất.

FLOORSMAX_MODE: Chế độ số tầng tối đa của các tòa nhà trong khu vực cư trú. Đây là số tầng tối đa phổ biến nhất.

FLOORSMIN_MODE: Chế độ số tầng tối thiểu của các tòa nhà trong khu vực cư trú. Đây là số tầng tối thiểu phổ biến nhất.

LANDAREA_MODE: Chế độ diện tích đất của các bất động sản. Đây là diện tích đất phổ biến nhất trong khu vực cư trú.

LIVINGAPARTMENTS_MODE: Chế độ số lượng căn hộ sinh sống trong các bất động sản. Đây là số lượng căn hộ sinh sống phổ biến nhất.

LIVINGAREA_MODE: Chế độ diện tích sinh sống trong các bất động sản. Đây là diện tích sinh sống phổ biến nhất.

NONLIVINGAPARTMENTS_MODE: Chế độ số lượng căn hộ không sinh sống trong các bất động sản. Đây là số lượng căn hộ không sinh sống phổ biến nhất.

NONLIVINGAREA_MODE: Chế độ diện tích không sinh sống trong các bất động sản. Đây là diện tích không sinh sống phổ biến nhất.

APARTMENTS_MEDI: Trung vị số lượng căn hộ trong các bất động sản. Đây là giá trị ở giữa khi các số lượng căn hộ được sắp xếp theo thứ tự.

BASEMENTAREA_MEDI: Trung vị diện tích tầng hầm của các bất động sản. Đây là diện tích tầng hầm ở giữa khi sắp xếp theo thứ tự.

YEARS_BEGINEXPLUATATION_MEDI: Trung vị số năm kể từ khi bất động sản bắt đầu được sử dụng. Đây là giá trị ở giữa khi số năm được sắp xếp theo thứ tự.

YEARS_BUILD_MEDI: Trung vị tuổi của các tòa nhà trong khu vực cư trú. Đây là số năm xây dựng ở giữa khi sắp xếp theo thứ tự.

COMMONAREA_MEDI: Trung vị diện tích khu vực chung trong các bất động sản. Đây là diện tích khu vực chung ở giữa khi sắp xếp theo thứ tự.

ELEVATORS_MEDI: Trung vị số lượng thang máy trong các tòa nhà. Đây là số lượng thang máy ở giữa khi sắp xếp theo thứ tự.

ENTRANCES_MEDI: Trung vị số lượng lối vào trong các tòa nhà. Đây là số lượng lối vào ở giữa khi sắp xếp theo thứ tự.

FLOORSMAX_MEDI: Trung vị số tầng tối đa của các tòa nhà trong khu vực cư trú. Đây là số tầng tối đa ở giữa khi sắp xếp theo thứ tự.

FLOORSMIN_MEDI: Trung vị số tầng tối thiểu của các tòa nhà trong khu vực cư trú. Đây là số tầng tối thiểu ở giữa khi sắp xếp theo thứ tự.

LANDAREA_MEDI: Trung vị diện tích đất của các bất động sản. Đây là diện tích đất ở giữa khi sắp xếp theo thứ tự.

LIVINGAPARTMENTS_MEDI: Trung vị số lượng căn hộ sinh sống trong các bất động sản. Đây là số lượng căn hộ sinh sống ở giữa khi sắp xếp theo thứ tự.

LIVINGAREA_MEDI: Trung vị diện tích sinh sống trong các bất động sản. Đây là diện tích sinh sống ở giữa khi sắp xếp theo thứ tự.

NONLIVINGAPARTMENTS_MEDI: Trung vị số lượng căn hộ không sinh sống trong các bất động sản. Đây là số lượng căn hộ không sinh sống ở giữa khi sắp xếp theo thứ tự.

NONLIVINGAREA_MEDI: Trung vị diện tích không sinh sống trong các bất động sản. Đây là diện tích không sinh sống ở giữa khi sắp xếp theo thứ tự.

FONDKAPREMONT_MODE: Chế độ của quỹ sửa chữa (ví dụ: "fund repair", "no repair", etc.). Cung cấp thông tin về loại quỹ bảo trì hoặc sửa chữa bất động sản.

HOUSETYPE_MODE: Chế độ loại hình nhà ở (ví dụ: "block of flats", "detached house", etc.). Cho biết loại nhà ở phổ biến nhất trong khu vực cư trú.

TOTALAREA_MODE: Chế độ diện tích tổng thể của bất động sản. Đây là diện tích tổng thể phổ biến nhất.

WALLSMATERIAL_MODE: Chế độ loại vật liệu xây dựng tường (ví dụ: "brick", "panel", etc.). Cung cấp thông tin về loại vật liệu xây dựng phổ biến nhất.

EMERGENCYSTATE_MODE: Chế độ trạng thái khẩn cấp (ví dụ: "no", "yes"). Cho biết tình trạng khẩn cấp phổ biến nhất liên quan đến bất động sản.

OBS_30_CNT_SOCIAL_CIRCLE: Số lượng lần mà khách hàng đã quan sát thấy các mối quan hệ xã hội trong vòng 30 ngày qua.

DEF_30_CNT_SOCIAL_CIRCLE: Số lượng lần mà khách hàng đã quan sát thấy các mối quan hệ xã hội gặp vấn đề về tài chính trong vòng 30 ngày qua (tức là có hành vi vỡ nợ hoặc trả chậm).

OBS_60_CNT_SOCIAL_CIRCLE: Số lượng lần mà khách hàng đã quan sát thấy các mối quan hệ xã hội trong vòng 60 ngày qua.

DEF_60_CNT_SOCIAL_CIRCLE: Số lượng lần mà khách hàng đã quan sát thấy các mối quan hệ xã hội gặp vấn đề về tài chính trong vòng 60 ngày qua.

DAYS_LAST_PHONE_CHANGE: Số ngày kể từ khi khách hàng thay đổi số điện thoại của mình. Thường là số âm (số ngày từ ngày thay đổi đến ngày hiện tại).

FLAG_DOCUMENT_2 đến FLAG_DOCUMENT_21: Các cột này là cờ cho biết liệu khách hàng đã cung cấp các tài liệu cụ thể hay không. Mỗi cột tương ứng với một loại tài liệu khác nhau (ví dụ: giấy tờ tùy thân, hợp đồng, chứng nhận thu nhập, v.v.). Giá trị là 1 nếu tài liệu đã được cung cấp, 0 nếu không.

AMT_REQ_CREDIT_BUREAU_HOUR: Tổng số tiền yêu cầu từ các cơ quan báo cáo tín dụng trong vòng một giờ.

AMT_REQ_CREDIT_BUREAU_DAY: Tổng số tiền yêu cầu từ các cơ quan báo cáo tín dụng trong vòng một ngày.

AMT_REQ_CREDIT_BUREAU_WEEK: Tổng số tiền yêu cầu từ các cơ quan báo cáo tín dụng trong vòng một tuần.

AMT_REQ_CREDIT_BUREAU_MON: Tổng số tiền yêu cầu từ các cơ quan báo cáo tín dụng trong vòng một tháng.

AMT_REQ_CREDIT_BUREAU_QRT: Tổng số tiền yêu cầu từ các cơ quan báo cáo tín dụng trong vòng một quý.

AMT_REQ_CREDIT_BUREAU_YEAR: Tổng số tiền yêu cầu từ các cơ quan báo cáo tín dụng trong vòng một năm.

# **CHECKING**

In [39]:
data

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.0220,0.0198,0.0,0.0000,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.0000,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.0790,0.0554,0.0,0.0000,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.0100,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [40]:
for i in data.columns:
  print(i)

SK_ID_CURR
TARGET
NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
CNT_CHILDREN
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
REGION_POPULATION_RELATIVE
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH
OWN_CAR_AGE
FLAG_MOBIL
FLAG_EMP_PHONE
FLAG_WORK_PHONE
FLAG_CONT_MOBILE
FLAG_PHONE
FLAG_EMAIL
OCCUPATION_TYPE
CNT_FAM_MEMBERS
REGION_RATING_CLIENT
REGION_RATING_CLIENT_W_CITY
WEEKDAY_APPR_PROCESS_START
HOUR_APPR_PROCESS_START
REG_REGION_NOT_LIVE_REGION
REG_REGION_NOT_WORK_REGION
LIVE_REGION_NOT_WORK_REGION
REG_CITY_NOT_LIVE_CITY
REG_CITY_NOT_WORK_CITY
LIVE_CITY_NOT_WORK_CITY
ORGANIZATION_TYPE
EXT_SOURCE_1
EXT_SOURCE_2
EXT_SOURCE_3
APARTMENTS_AVG
BASEMENTAREA_AVG
YEARS_BEGINEXPLUATATION_AVG
YEARS_BUILD_AVG
COMMONAREA_AVG
ELEVATORS_AVG
ENTRANCES_AVG
FLOORSMAX_AVG
FLOORSMIN_AVG
LANDAREA_AVG
LIVINGAPARTMENTS_AVG
LIVINGAREA_AVG
NONLIVINGAPARTMENTS_AVG
NONLIVINGAREA_AVG
APART

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


***Kiểm tra xem có bao nhiêu columns có giá trị Inf hoặc Null không. Đối với các columns có Null cao, xóa bớt đi***

In [42]:
numerical_df = data[data.select_dtypes(exclude='object').columns]

inf_mask = np.isinf(numerical_df)
has_inf = inf_mask.any().any()
print(f"DataFrame có giá trị vô cực: {has_inf}")

DataFrame có giá trị vô cực: False


In [43]:
null_cols_to_droped = []
for i in data.columns:
  if data[i].isnull().sum() >= int(data.shape[0]//2):
    null_cols_to_droped.append(i)
  else:
    continue

print(len(null_cols_to_droped))
print(null_cols_to_droped)

41
['OWN_CAR_AGE', 'EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE']


In [44]:
data = data.drop(columns=null_cols_to_droped)

***Kiểm tra các giá trị bất thường:***

*Như vậy, có thể thấy data không có giá trị bất thường*

In [45]:
data.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,FLOORSMAX_MODE,YEARS_BEGINEXPLUATATION_MEDI,FLOORSMAX_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307509.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,3.068510e+05,246546.000000,157504.000000,154491.000000,157504.000000,154491.000000,157504.000000,154491.000000,159080.000000,306490.000000,306490.000000,306490.000000,306490.000000,307510.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.00000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,-4986.120328,-2994.202373,0.999997,0.819889,0.199368,0.998133,0.281066,0.056720,2.152665,2.052463,2.031521,12.063419,0.015144,0.050769,0.040659,0.078173,0.230454,0.179555,5.143927e-01,0.510853,0.977735,0.226282,0.977065,0.222315,0.977752,0.225897,0.102547,1.422245,0.143421,1.405292,0.100049,-962.858788,0.000042,0.710023,0.000081,0.015115,0.088055,0.000192,0.081376,0.003896,0.000023,0.003912,0.000007,0.003525,0.002936,0.00121,0.009928,0.000267,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,3522.886321,1509.450419,0.001803,0.384280,0.399526,0.043164,0.449521,0.231307,0.910682,0.509034,0.502737,3.265832,0.122126,0.219526,0.197499,0.268444,0.421124,0.383817,1.910602e-01,0.194844,0.059223,0.144641,0.064575,0.143709,0.059897,0.145067,0.107462,2.400989,0.446698,2.379803,0.362291,826.808487,0.006502,0.453752,0.009016,0.122010,0.283376,0.013850,0.273412,0.062295,0.004771,0.062424,0.002550,0.059268,0.054110,0.03476,0.099144,0.016327,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,-24672.000000,-7197.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.173617e-08,0.000527,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4292.000000,0.000000,0.000000,0

In [46]:
#trong Column "DAY_EMPLOYED"có giá trị bất thường là 365243 (quá lớn). Chuyển đổi các giá trị này thành Null
data.loc[data['DAYS_EMPLOYED'] == 365243, 'DAYS_EMPLOYED'] = np.nan

***Tiếp theo, phân bổ các columns còn lại vào 2 loại: Các columns phân loại (Category) và Columns số (Numerics) để phục vụ phân tích sau***

In [47]:
def classify_cols(data):
  obj_cols = data.select_dtypes(include='object').columns
  num_cols = data.select_dtypes(exclude='object').columns
  category_variables = obj_cols.tolist()
  numberic_variables = []

  for i in num_cols:
    if len(data[i].unique()) <30:
      category_variables.append(i)
    else:
      numberic_variables.append(i)

  print("Numbers of category columns:", len(category_variables))
  print(category_variables)
  print('\n')
  print("Numbers of numberic columns:", len(numberic_variables))
  print(numberic_variables)
  return category_variables, numberic_variables

category_variables, numberic_variables = classify_cols(data)

Numbers of category columns: 60
['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'EMERGENCYSTATE_MODE', 'TARGET', 'CNT_CHILDREN', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLOORSMAX_MODE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_

***Kiểm tra xem trong 2 loại columns Categori và Numeric, có bao nhiêu columns chứa Null, qua đó sẽ xử lý bằng cách thêm dữ liệu***

In [48]:
null_category_variables = []
null_numberic_variables = []
for i in category_variables:
  if data[i].isnull().sum() > 0:
    null_category_variables.append(i)

for i in numberic_variables:
  if data[i].isnull().sum() > 0:
    null_numberic_variables.append(i)

print("Numbers of null category columns:", len(null_category_variables))
print(null_category_variables)
print('\n')
print("Numbers of null numberic columns:", len(null_numberic_variables))
print(null_numberic_variables)

Numbers of null category columns: 13
['NAME_TYPE_SUITE', 'OCCUPATION_TYPE', 'EMERGENCYSTATE_MODE', 'CNT_FAM_MEMBERS', 'FLOORSMAX_MODE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']


Numbers of null numberic columns: 14
['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_EMPLOYED', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'YEARS_BEGINEXPLUATATION_AVG', 'FLOORSMAX_AVG', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BEGINEXPLUATATION_MEDI', 'FLOORSMAX_MEDI', 'TOTALAREA_MODE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE']


In [49]:
#Hàm này sẽ chuyển các giá trị NaN của các columns số thành giá trị trung bình,...
#còn các columns Phân loại thì thay thế bằng giá trị chiếm đa số
def fillna(data):
    for col in null_numberic_variables:
        data[col] = data[col].fillna(data[col].median())
    for col in null_category_variables:
        data[col] = data[col].fillna(data[col].mode()[0])

fillna(data)

***Sau khi dữ liệu đã hết Null, tiến hành Xử lý dữ liệu Ouliers:***

In [50]:
#Skewness cho các biến liên tục hoặc rời rạc:
for i in numberic_variables:
  skewness = skew(data[i])
  print(f"Skewness của cột {i}:", skewness)

Skewness của cột SK_ID_CURR: -0.001200229222216958
Skewness của cột AMT_INCOME_TOTAL: 391.55774412283097
Skewness của cột AMT_CREDIT: 1.234772474032497
Skewness của cột AMT_ANNUITY: 1.57981601850358
Skewness của cột AMT_GOODS_PRICE: 1.3501359638005384
Skewness của cột REGION_POPULATION_RELATIVE: 1.4880012623794292
Skewness của cột DAYS_BIRTH: -0.11567274746102389
Skewness của cột DAYS_EMPLOYED: -2.295837876788367
Skewness của cột DAYS_REGISTRATION: -0.5908687335336363
Skewness của cột DAYS_ID_PUBLISH: 0.349325789225092
Skewness của cột EXT_SOURCE_2: -0.7959361537903271
Skewness của cột EXT_SOURCE_3: -0.537694074094104
Skewness của cột YEARS_BEGINEXPLUATATION_AVG: -21.744659816741787
Skewness của cột FLOORSMAX_AVG: 2.300258467827182
Skewness của cột YEARS_BEGINEXPLUATATION_MODE: -20.686068062184354
Skewness của cột YEARS_BEGINEXPLUATATION_MEDI: -21.825280491207565
Skewness của cột FLOORSMAX_MEDI: 2.315259061106669
Skewness của cột TOTALAREA_MODE: 4.216459182974029
Skewness của cột OBS_3

In [51]:
#Z-score cho các columns phân phối chuẩn:
z_score_list = ['DAYS_LAST_PHONE_CHANGE', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_ID_PUBLISH', 'DAYS_REGISTRATION', 'DAYS_BIRTH']
threshold = 3  # Xác định ngưỡng cho outlier

for i in z_score_list:
  data.loc[:, 'z_score'] = zscore(data[i])   # Tính Z-score cho cột hiện tại
  data = data[(data['z_score'] <= threshold) & (data['z_score'] >= -threshold)]   # Lọc và loại bỏ các giá trị ngoại lệ

# Loại bỏ cột 'z_score' sau khi lọc
data = data.drop(columns=['z_score'])

print("Dữ liệu sau khi loại bỏ các ngoại lệ bằng phương pháp Z-score:")
print(data.shape)

Dữ liệu sau khi loại bỏ các ngoại lệ bằng phương pháp Z-score:
(306123, 81)


# **PREPROCESSING**

***Pearson Correlation để kiểm tra độ tuyến tính:***

*Hầu hết các columns đều có giá trị Corr thấp, nên mô hình thiên về phi tuyến*

In [52]:
pearson_cols = numberic_variables

for i in pearson_cols:
  corr, _ = pearsonr(data[i], data['TARGET'])
  print(f"Pearson Correlation of column {i}:", corr)

Pearson Correlation of column SK_ID_CURR: -0.0019758973004831283
Pearson Correlation of column AMT_INCOME_TOTAL: -0.003961089259600312
Pearson Correlation of column AMT_CREDIT: -0.03070677862488917
Pearson Correlation of column AMT_ANNUITY: -0.01293253809299193
Pearson Correlation of column AMT_GOODS_PRICE: -0.03995717541708664
Pearson Correlation of column REGION_POPULATION_RELATIVE: -0.03695522664521479
Pearson Correlation of column DAYS_BIRTH: 0.07791880770460743
Pearson Correlation of column DAYS_EMPLOYED: 0.06359474292131911
Pearson Correlation of column DAYS_REGISTRATION: 0.041250051709563876
Pearson Correlation of column DAYS_ID_PUBLISH: 0.051282342459133476
Pearson Correlation of column EXT_SOURCE_2: -0.16054037048318187
Pearson Correlation of column EXT_SOURCE_3: -0.15584637439764537
Pearson Correlation of column YEARS_BEGINEXPLUATATION_AVG: -0.004692543689934184
Pearson Correlation of column FLOORSMAX_AVG: -0.03939960157658874
Pearson Correlation of column YEARS_BEGINEXPLUATA

***Kiểm định Chi-squared cho các columns phân loại:***

In [53]:
remove_obj_cols =[]

#Chi-square test cho các cột Object so với cột Output của data
for i in category_variables:
  chi2, pval, _, _ = chi2_contingency(pd.crosstab(data[i], data['TARGET']))
  print(i, '---', pval)
  if pval > 0.05:
    remove_obj_cols.append(i)
  else:
    continue
print(remove_obj_cols)

NAME_CONTRACT_TYPE --- 8.67198620644725e-66
CODE_GENDER --- 1.6110806745796115e-200
FLAG_OWN_CAR --- 1.891623883695622e-33
FLAG_OWN_REALTY --- 0.0009162510255909044
NAME_TYPE_SUITE --- 1.5124421452865175e-05
NAME_INCOME_TYPE --- 1.5101656705221008e-264
NAME_EDUCATION_TYPE --- 8.745199943966637e-219
NAME_FAMILY_STATUS --- 1.2980403359314118e-105
NAME_HOUSING_TYPE --- 2.999628472060348e-88
OCCUPATION_TYPE --- 6.165487872390468e-242
WEEKDAY_APPR_PROCESS_START --- 0.023523193544129753
ORGANIZATION_TYPE --- 1.2151153699574255e-296
EMERGENCYSTATE_MODE --- 0.013143046740033632
TARGET --- 0.0
CNT_CHILDREN --- 1.8772016669209422e-31
FLAG_MOBIL --- 1.0
FLAG_EMP_PHONE --- 1.054325110864873e-141
FLAG_WORK_PHONE --- 2.0394825859236082e-55
FLAG_CONT_MOBILE --- 0.9147395516024521
FLAG_PHONE --- 4.3946819851346255e-39
FLAG_EMAIL --- 0.33939562577591065
CNT_FAM_MEMBERS --- 3.2517528587391694e-27
REGION_RATING_CLIENT --- 8.696857942696058e-230
REGION_RATING_CLIENT_W_CITY --- 2.703475671494699e-246
HOUR_

In [54]:
data = data.drop(columns = remove_obj_cols)

***Kiểm định Spearman cho các columns số:***

In [55]:
spear_cols = numberic_variables
spear_cols.append('TARGET')
spearman_df = data[spear_cols]
correlation_results = []

# Duyệt qua tất cả các cột trong DataFrame
for col in spearman_df.columns:
  if col == 'TARGET':
    continue
  else:
    corr, p_value = spearmanr(spearman_df[col], spearman_df['TARGET'])
    correlation_results.append((col, corr, p_value))

# Chuyển kết quả thành DataFrame để dễ quan sát
correlation_df = pd.DataFrame(correlation_results, columns=['Feature', 'Spearman Correlation', 'p-value'])
correlation_df = correlation_df.sort_values(by='Spearman Correlation', ascending=False)
correlation_df

,Feature,Spearman Correlation,p-value
6,DAYS_BIRTH,0.078015,0.000000e+00
7,DAYS_EMPLOYED,0.075069,0.000000e+00
20,DAYS_LAST_PHONE_CHANGE,0.053554,3.224461e-193
9,DAYS_ID_PUBLISH,0.052370,7.540553e-185
8,DAYS_REGISTRATION,0.039596,1.811763e-106
18,OBS_30_CNT_SOCIAL_CIRCLE,0.009681,8.482638e-08
19,OBS_60_CNT_SOCIAL_CIRCLE,0.009530,1.342328e-07
3,AMT_ANNUITY,-0.000153,9.325258e-01
0,SK_ID_CURR,-0.001978,2.737096e-01
14,YEARS_BEGINEXPLUATATION_MODE,-0.016597,4.172014e-20


In [56]:
data = data.drop(columns = ['OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'AMT_ANNUITY', 'SK_ID_CURR',
                            'YEARS_BEGINEXPLUATATION_MODE', 'AMT_CREDIT', 'YEARS_BEGINEXPLUATATION_MEDI',
                            'YEARS_BEGINEXPLUATATION_AVG', 'AMT_INCOME_TOTAL'])

# **EDA**

In [57]:
data = data[data['CODE_GENDER']!='XNA']
category_variables, numberic_variables = classify_cols(data)

Numbers of category columns: 44
['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'EMERGENCYSTATE_MODE', 'TARGET', 'CNT_CHILDREN', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLOORSMAX_MODE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_18', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']


N

***Biểu đồ Bar cho các columns phân loại***

In [ ]:
for i in category_variables:
  print(data[i].value_counts())
  print('\n')

NAME_CONTRACT_TYPE
Cash loans         276933
Revolving loans     29186
Name: count, dtype: int64


CODE_GENDER
F    201448
M    104671
Name: count, dtype: int64


FLAG_OWN_CAR
N    201962
Y    104157
Name: count, dtype: int64


FLAG_OWN_REALTY
Y    212319
N     93800
Name: count, dtype: int64


NAME_TYPE_SUITE
Unaccompanied      248671
Family              39975
Spouse, partner     11331
Children             3250
Other_B              1758
Other_A               863
Group of people       271
Name: count, dtype: int64


NAME_INCOME_TYPE
Working                 158342
Commercial associate     71292
Pensioner                54801
State servant            21629
Unemployed                  22
Student                     18
Businessman                 10
Maternity leave              5
Name: count, dtype: int64


NAME_EDUCATION_TYPE
Secondary / secondary special    217444
Higher education                  74472
Incomplete higher                 10253
Lower secondary                    3788
Acade

In [ ]:
for i in category_variables:
    ax1 = sns.countplot(x=i, hue='TARGET', data=data)
    ax1.set_xlabel('Value:')
    ax1.set_ylabel('Number')
    ax1.set_title(i)
    plt.gcf().set_size_inches(len(data[i].unique()) * 2.1, 4)

    for container in ax1.containers:
        ax1.bar_label(container)

    plt.tight_layout()
    plt.show()

***Biểu đồ Hist cho các columns số***

In [ ]:
for i in numberic_variables:
  plt.figure(figsize=(10,5))
  ax2 = sns.histplot(data[i], bins=20)
  ax2.set_xlabel('Value')
  ax2.set_ylabel('Frequency')
  ax2.set_title(i)
  plt.show()

# **ENCODING**

In [20]:
#Ánh xạ các giá trị của NAME_HOUSING_TYPE
data.loc[:, 'NAME_HOUSING_TYPE'] = data['NAME_HOUSING_TYPE'].apply(lambda x: 1 if x == 'House / apartment' else 0)

In [21]:
#Ánh xạ các giá trị của OCCUPATION_TYPE
OCCUPATION_TYPE_mapping = {'Laborers': 0, 'Core staff': 2, 'Accountants': 1, 'Managers': 2, 'Drivers': 0, 'Sales staff': 1,
                           'Cleaning staff': 0, 'Cooking staff': 0, 'Private service staff': 0, 'Medicine staff': 0,
                           'Security staff': 0, 'High skill tech staff': 1, 'Waiters/barmen staff': 0, 'Low-skill Laborers': 0,
                           'Realty agents': 2, 'Secretaries': 1, 'IT staff': 1, 'HR staff': 1}
data.loc[:,'OCCUPATION_TYPE'] = data['OCCUPATION_TYPE'].map(OCCUPATION_TYPE_mapping)

In [22]:
data.loc[:, ['NAME_HOUSING_TYPE', 'OCCUPATION_TYPE']] = data[['NAME_HOUSING_TYPE', 'OCCUPATION_TYPE']].astype('int64')

In [23]:
def map_columns(data, column_names=['NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
                                    'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'EMERGENCYSTATE_MODE']):

    # Các ánh xạ giá trị cho các cột khác nhau
    NAME_TYPE_SUITE_mapping = {'Unaccompanied': 0, 'Family': 3, 'Spouse, partner': 2, 'Children': 1, 'Other_B': 4, 'Other_A': 4, 'Group of people': 4}

    NAME_INCOME_TYPE_mapping = {'Working': 2, 'Commercial associate': 3, 'Pensioner': 1, 'State servant': 3, 'Unemployed': 0,
                                'Student': 0, 'Businessman': 3, 'Maternity leave': 1}

    NAME_EDUCATION_TYPE_mapping = {'Secondary / secondary special': 1, 'Higher education': 3, 'Incomplete higher': 2,
                                   'Lower secondary': 0, 'Academic degree': 4}

    NAME_FAMILY_STATUS_mapping = {'Married': 1, 'Single / not married': 0, 'Civil marriage': 0, 'Separated': 0, 'Widow': 0, 'Unknown': 0}

    WEEKDAY_APPR_PROCESS_START_mapping = {'TUESDAY': 1, 'WEDNESDAY': 2, 'MONDAY': 0, 'THURSDAY': 3, 'FRIDAY': 4, 'SATURDAY': 5, 'SUNDAY': 6}

    ORGANIZATION_TYPE_mapping = {'Business Entity Type 3': 1, 'XNA': 0, 'Self-employed': 1, 'Other': 0, 'Medicine': 2, 'Business Entity Type 2': 1,
                                 'Government': 2, 'School': 2, 'Trade: type 7': 1, 'Kindergarten': 2, 'Construction': 2, 'Business Entity Type 1': 1,
                                 'Transport: type 4': 2, 'Trade: type 3': 1, 'Industry: type 9': 2, 'Industry: type 3': 2, 'Security': 2, 'Housing': 1,
                                 'Industry: type 11': 1, 'Military': 2, 'Bank': 1, 'Agriculture': 2, 'Police': 2, 'Transport: type 2': 2, 'Postal': 2,
                                 'Security Ministries': 2, 'Trade: type 2': 1, 'Restaurant': 1, 'Services': 1, 'University': 2, 'Industry: type 7': 2,
                                 'Transport: type 3': 2, 'Industry: type 1': 2, 'Hotel': 1, 'Electricity': 2, 'Industry: type 4': 2, 'Trade: type 6': 1,
                                 'Insurance': 1, 'Industry: type 5': 2, 'Telecom': 2, 'Emergency': 2, 'Industry: type 2': 2, 'Advertising': 1, 'Realtor': 1,
                                 'Culture': 2, 'Industry: type 12': 2, 'Trade: type 1': 1, 'Mobile': 2, 'Legal Services': 2, 'Cleaning': 2, 'Transport: type 1': 2,
                                 'Industry: type 6': 2, 'Industry: type 10': 2, 'Religion': 2,  'Industry: type 13': 2, 'Trade: type 4': 1, 'Trade: type 5': 1,
                                 'Industry: type 8': 2}

    EMERGENCYSTATE_MODE_mapping = {'No': 0, 'Yes': 1}


    # Thực hiện ánh xạ cho từng cột trong DataFrame
    mappings = {'NAME_TYPE_SUITE': NAME_TYPE_SUITE_mapping, 'NAME_INCOME_TYPE': NAME_INCOME_TYPE_mapping,
                'NAME_EDUCATION_TYPE': NAME_EDUCATION_TYPE_mapping, 'NAME_FAMILY_STATUS': NAME_FAMILY_STATUS_mapping,
                'WEEKDAY_APPR_PROCESS_START': WEEKDAY_APPR_PROCESS_START_mapping, 'ORGANIZATION_TYPE': ORGANIZATION_TYPE_mapping,
                'EMERGENCYSTATE_MODE': EMERGENCYSTATE_MODE_mapping}

    for column in column_names:
        if column in data.columns and column in mappings:
            data.loc[:, column] = data[column].map(mappings[column])
            data[column] = data[column].astype('int64')

    return data
data = map_columns(data)

In [24]:
#One hot coding cho các columns Category còn lại:
OHC_cols = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'EMERGENCYSTATE_MODE']
data = pd.get_dummies(data, columns = OHC_cols, drop_first=True)

In [25]:
data

,TARGET,CNT_CHILDREN,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_2,EXT_SOURCE_3,FLOORSMAX_AVG,FLOORSMAX_MODE,FLOORSMAX_MEDI,TOTALAREA_MODE,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_11,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_18,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,EMERGENCYSTATE_MODE_1
0,1,0,351000.0,0,2,1,0,1,0.018801,-9461,-637.0,-3648.0,-2120,1,0,1,0,1.0,2,2,2,10,0,0,0,0,0,1,0.262949,0.139376,0.0833,0.0833,0.0833,0.0149,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,False,True,False,True,False
1,0,0,1129500.0,3,3,3,1,1,0.003541,-16765,-1188.0,-1186.0,-291,1,0,1,2,2.0,1,1,0,11,0,0,0,0,0,2,0.622246,0.535276,0.2917,0.2917,0.2917,0.0714,0.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,False,False,False,False,False
2,0,0,135000.0,0,2,1,0,1,0.010032,-19046,-225.0,-4260.0,-2531,1,1,1,0,1.0,2,2,0,9,0,0,0,0,0,2,0.555912,0.729567,0.1667,0.1667,0.1667,0.0688,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,True,True,True,True,False
3,0,0,297000.0,0,2,1,0,1,0.008019,-19005,-3039.0,-9833.0,-2437,1,0,0,0,2.0,2,2,2,17,0,0,0,0,0,1,0.650442,0.535276,0.1667,0.1667,0.1667,0.0688,0.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,False,False,False,True,False
4,0,0,513000.0,0,2,1,0,1,0.028663,-19932,-3038.0,-4311.0,-3458,1,0,0,2,1.0,2,2,3,11,0,0,0,1,1,2,0.322738,0.535276,0.1667,0.1667,0.1667,0.0688,0.0,0.0,-1106.0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,0,225000.0,0,2,1,0,0,0.032561,-9327,-236.0,-8456.0,-1982,1,0,0,1,1.0,1,1,3,15,0,0,0,0,0,1,0.681632,0.535276,0.6042,0.4583,0.6042,0.2898,0.0,0.0,-273.0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,False,True,False,False,False
307507,0,0,225000.0,0,1,1,0,1,0.025164,-20775,-1648.0,-4388.0,-4090,0,0,1,0,1.0,2,2,0,8,0,0,0,0,0,0,0.115992,0.535276,0.0833,0.0833,0.0833,0.0214,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,False,False,False,True,False
307508,0,0,585000.0,0,2,3,0,1,0.005002,-14966,-7921.0,-6737.0,-5150,1,0,0,2,1.0,3,3,3,9,0,0,0,1,1,2,0.535722,0.218859,0.1667,0.1667,0.1667,0.7970,0.0,0.0,-1909.0,0,1,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,False,False,False,True,False
307509,1,0,319500.0,0,3,1,1,1,0.005313,-11961,-4786.0,-2562.0,-931,1,0,0,0,2.0,2,2,2,9,0,0,1,1,0,1,0.514163,0.661024,0.0417,0.0417,0.0417,0.0086,0.0,0.0,-322.0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,False,False,False,True,False


# **PREDICTION**

In [26]:
#Hàm đánh giá hiệu suất
def print_scores(y_test, y_pred):
  print(classification_report(y_test, y_pred))

#Hàm chia tập train-test và MinMax Scaler
def preprocess_data(data, test_size=0.2, random_state=42):

    # Chia tập train-test
    X = data.drop(columns=['TARGET'], axis=1)
    features = X.columns

    y = data['TARGET']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    #MinMax Scaler
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocess_data(data)

***1.Random Forest:*** *Overfitting*

In [ ]:
model_RF = RandomForestClassifier(n_estimators=200, random_state=42, class_weight = 'balanced')
model_RF.fit(X_train, y_train)

y_train_pred = model_RF.predict(X_train)
y_test_pred = model_RF.predict(X_test)

print_scores(y_train, y_train_pred)
print_scores(y_test, y_test_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    225077
           1       1.00      1.00      1.00     19818

    accuracy                           1.00    244895
   macro avg       1.00      1.00      1.00    244895
weighted avg       1.00      1.00      1.00    244895

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56275
           1       0.65      0.00      0.01      4949

    accuracy                           0.92     61224
   macro avg       0.79      0.50      0.48     61224
weighted avg       0.90      0.92      0.88     61224



***2.XGBOOST:*** *Có phần cải thiện hơn đôi chút, nhưng vẫn thấp*

In [ ]:
model_XGB = xgb.XGBClassifier(n_estimators=200, random_state=42)
model_XGB.fit(X_train, y_train)

y_train_pred = model_XGB.predict(X_train)
y_test_pred = model_XGB.predict(X_test)

print_scores(y_train, y_train_pred)
print_scores(y_test, y_test_pred)

              precision    recall  f1-score   support

           0       0.93      1.00      0.96    225077
           1       0.91      0.11      0.19     19818

    accuracy                           0.93    244895
   macro avg       0.92      0.55      0.58    244895
weighted avg       0.93      0.93      0.90    244895

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56275
           1       0.43      0.03      0.06      4949

    accuracy                           0.92     61224
   macro avg       0.68      0.51      0.51     61224
weighted avg       0.88      0.92      0.88     61224



***3.SMOTE tích hợp XGBOOST: *** *Không có cải thiện thêm*

In [ ]:
# Áp dụng SMOTE để tăng cường lớp 1
class_counts = y_train.value_counts()
smote = SMOTE(sampling_strategy={1: class_counts[0]}, random_state=42)
X_train_resample, y_train_resample = smote.fit_resample(X_train, y_train)

In [ ]:
model_XGB_2 = xgb.XGBClassifier(n_estimators=200, random_state=42)
model_XGB_2.fit(X_train_resample, y_train_resample)

y_train_pred = model_XGB_2.predict(X_train_resample)
y_test_pred = model_XGB_2.predict(X_test)

print_scores(y_train_resample, y_train_pred)
print_scores(y_test, y_test_pred)

              precision    recall  f1-score   support

           0       0.92      1.00      0.96    225077
           1       1.00      0.92      0.96    225077

    accuracy                           0.96    450154
   macro avg       0.96      0.96      0.96    450154
weighted avg       0.96      0.96      0.96    450154

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56275
           1       0.41      0.04      0.07      4949

    accuracy                           0.92     61224
   macro avg       0.67      0.52      0.51     61224
weighted avg       0.88      0.92      0.89     61224



***4.UNDER SAMPLING:*** *Tốt hơn hẳn, có thể xem xét cải thiện*

In [27]:
df0 = data[data['TARGET']==0]
df1 = data[data['TARGET']==1]

In [33]:
#Kiểm định IQR cho các columns của df0:
IQR_list = ['DAYS_EMPLOYED', 'FLOORSMAX_AVG', 'FLOORSMAX_MEDI', 'TOTALAREA_MODE']

for i in IQR_list:
  #tính toán IQR thông qua Q1,
  Q1 = df0[i].quantile(0.25)
  Q3 = df0[i].quantile(0.75)
  IQR = Q3 - Q1

  # Xác định ngưỡng cho giá trị ngoại lệ
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  # Nếu bạn muốn loại bỏ các giá trị ngoại lệ
  df0 = df0[(df0[i] >= lower_bound) & (df0[i] <= upper_bound)]

print("Dữ liệu sau khi loại bỏ Outliers sử dụng IQR:")
print(df0.shape)

Dữ liệu sau khi loại bỏ Outliers sử dụng IQR:
(124866, 56)


In [34]:
df0_under = df0.sample(30000, random_state=42)

In [35]:
data_under = pd.concat([df0_under, df1])
data_under = data_under.sample(frac=1, random_state=42)

In [36]:
X_train_under, X_test_under, y_train_under, y_test_under = preprocess_data(data_under)

In [37]:
model_XGB_3 = xgb.XGBClassifier(n_estimators=200, random_state=42)
model_XGB_3.fit(X_train_under, y_train_under)

y_train_pred = model_XGB_3.predict(X_train_under)
y_test_pred = model_XGB_3.predict(X_test_under)

print_scores(y_train_under, y_train_pred)
print_scores(y_test_under, y_test_pred)

              precision    recall  f1-score   support

           0       0.87      0.97      0.92     24006
           1       0.96      0.83      0.89     19807

    accuracy                           0.90     43813
   macro avg       0.91      0.90      0.90     43813
weighted avg       0.91      0.90      0.90     43813

              precision    recall  f1-score   support

           0       0.75      0.87      0.80      5994
           1       0.80      0.65      0.72      4960

    accuracy                           0.77     10954
   macro avg       0.78      0.76      0.76     10954
weighted avg       0.77      0.77      0.77     10954



***5.Tối ưu hóa bằng UNDERSAMPLING vs CATBOOST:*** *Không cải thiện so với Under*

In [ ]:
def objective(trial):
  model_CB = CatBoostClassifier(
    iterations=trial.suggest_int("iterations", 100, 1000),
    learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
    depth=trial.suggest_int("depth", 4, 10),
    boosting_type=trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
    bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian", "MVS"]),
    l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
    random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
    bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
    od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
    od_wait=trial.suggest_int("od_wait", 10, 50),
    min_data_in_leaf=trial.suggest_int("min_data_in_leaf", 51, 120),
    loss_function='Logloss',
    eval_metric='Logloss',
    task_type='CPU',  # Chạy trên CPU thay vì GPU
    verbose=False,
)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-08-25 15:39:20,740] A new study created in memory with name: no-name-a209748a-c4b3-48b4-8213-3717ca8b8dee
[W 2024-08-25 15:39:20,749] Trial 0 failed with parameters: {'iterations': 344, 'learning_rate': 0.04788300130721402, 'depth': 4, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'l2_leaf_reg': 0.00204860845522725, 'random_strength': 0.0005982358475085985, 'bagging_temperature': 2.4996278129554863, 'od_type': 'IncToDec', 'od_wait': 19, 'min_data_in_leaf': 109} because of the following error: The value None could not be cast to float..
[W 2024-08-25 15:39:20,751] Trial 0 failed with value None.
[W 2024-08-25 15:39:20,756] Trial 1 failed with parameters: {'iterations': 825, 'learning_rate': 0.030589673753801276, 'depth': 6, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'l2_leaf_reg': 9.286130181904198e-06, 'random_strength': 4.07361694630295e-05, 'bagging_temperature': 0.8247374101647853, 'od_type': 'IncToDec', 'od_wait': 23, 'min_data_in_leaf': 96} because of t

In [ ]:
best_params = {'iterations': 277, 'learning_rate': 0.0035690005606886224, 'depth': 8, 'boosting_type': 'Ordered',
               'bootstrap_type': 'MVS', 'l2_leaf_reg': 4.630242174115198e-05, 'random_strength': 1.9086788208321497,
               'bagging_temperature': 2.6585254414439854, 'od_type': 'IncToDec', 'od_wait': 41, 'min_data_in_leaf': 91}

In [ ]:
# Huấn luyện mô hình với hyperparameters tốt nhất
best_params = best_params
model_CB = CatBoostClassifier(**best_params)
model_CB.fit(X_train_under, y_train_under)

probs = model_CB.predict_proba(X_test_under)
y_train_pred = model_CB.predict(X_train_under)
y_test_pred = model_CB.predict(X_test_under)

print(probs)
print(classification_report(y_train_under, y_train_pred))
print(classification_report(y_test_under, y_test_pred))

0:	learn: 0.6926178	total: 374ms	remaining: 1m 43s
1:	learn: 0.6920894	total: 699ms	remaining: 1m 36s
2:	learn: 0.6915763	total: 1.01s	remaining: 1m 32s
3:	learn: 0.6911062	total: 1.29s	remaining: 1m 27s
4:	learn: 0.6905966	total: 1.59s	remaining: 1m 26s
5:	learn: 0.6900796	total: 1.9s	remaining: 1m 26s
6:	learn: 0.6895753	total: 2.22s	remaining: 1m 25s
7:	learn: 0.6891369	total: 2.53s	remaining: 1m 24s
8:	learn: 0.6887088	total: 2.82s	remaining: 1m 23s
9:	learn: 0.6882185	total: 3s	remaining: 1m 20s
10:	learn: 0.6877515	total: 3.17s	remaining: 1m 16s
11:	learn: 0.6872469	total: 3.34s	remaining: 1m 13s
12:	learn: 0.6867863	total: 3.5s	remaining: 1m 11s
13:	learn: 0.6862855	total: 3.7s	remaining: 1m 9s
14:	learn: 0.6857964	total: 3.87s	remaining: 1m 7s
15:	learn: 0.6853712	total: 4.04s	remaining: 1m 5s
16:	learn: 0.6848835	total: 4.21s	remaining: 1m 4s
17:	learn: 0.6843855	total: 4.38s	remaining: 1m 3s
18:	learn: 0.6839118	total: 4.55s	remaining: 1m 1s
19:	learn: 0.6834653	total: 4.73s	

***6.GRIDSEARCH***

In [ ]:
param_grid = {
  'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 0.9],
  'learning_rate'   : [0.001, 0.01, 0.1, 1],
  'max_depth'       : [3, 5, 8, 10],
  'alpha'           : [1, 10, 100],
  'n_estimators'    : [10, 50, 100]
}

index = 0

answers_grid = {
    'combination'       :[],
    'train_Accuracy'    :[],
    'test_Accuracy'     :[],
    'colsample_bytree'  :[],
    'learning_rate'     :[],
    'max_depth'         :[],
    'alpha'             :[],
    'n_estimators'      :[]
}

In [ ]:
# Tạo một lưới các tham số
param_list = list(ParameterGrid(param_grid))

for params in param_list:
    # Tạo mô hình với các tham số hiện tại
    model_XGB_new = xgb.XGBClassifier(
        objective='binary:logistic',  # Sử dụng mục tiêu phân loại nhị phân
        colsample_bytree=params['colsample_bytree'],
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
        alpha=params['alpha'],
        n_estimators=params['n_estimators']
    )

    # Huấn luyện mô hình
    model_XGB_new.fit(X_train_under, y_train_under)

    # Dự đoán trên tập huấn luyện và kiểm tra
    y_train_pred = model_XGB_new.predict(X_train_under)
    y_test_pred = model_XGB_new.predict(X_test_under)

    # Tính toán độ chính xác
    train_accuracy = accuracy_score(y_train_under, y_train_pred)
    test_accuracy = accuracy_score(y_test_under, y_test_pred)

    # Lưu trữ kết quả
    answers_grid['combination'].append(params)
    answers_grid['train_Accuracy'].append(train_accuracy)
    answers_grid['test_Accuracy'].append(test_accuracy)
    answers_grid['colsample_bytree'].append(params['colsample_bytree'])
    answers_grid['learning_rate'].append(params['learning_rate'])
    answers_grid['max_depth'].append(params['max_depth'])
    answers_grid['alpha'].append(params['alpha'])
    answers_grid['n_estimators'].append(params['n_estimators'])

# In kết quả tham số tốt nhất dựa trên độ chính xác tập kiểm tra
best_index = answers_grid['test_Accuracy'].index(max(answers_grid['test_Accuracy']))
print("Best Hyperparameters:", answers_grid['combination'][best_index])

print_scores(y_train_under, y_train_pred)
print_scores(y_test_under, y_test_pred)

Best Hyperparameters: {'alpha': 10, 'colsample_bytree': 0.3, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
              precision    recall  f1-score   support

           0       0.71      0.76      0.73     24006
           1       0.68      0.61      0.65     19807

    accuracy                           0.70     43813
   macro avg       0.69      0.69      0.69     43813
weighted avg       0.69      0.70      0.69     43813

              precision    recall  f1-score   support

           0       0.70      0.75      0.72      5994
           1       0.67      0.61      0.63      4960

    accuracy                           0.68     10954
   macro avg       0.68      0.68      0.68     10954
weighted avg       0.68      0.68      0.68     10954



In [ ]:
# Tạo confusion_matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)
sns.heatmap(conf_matrix, xticklabels=["Not Fraud", "Fraud"], yticklabels=['Not Fraud', 'Fraud'], annot=True, fmt="d")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Values")
plt.ylabel("True Values")
plt.show()